## Dynamic Plans

This notebook implements a uav logic instead of a plan and decisions are done online following that logic

In [1]:
import numpy as np

# Custom modules
from helpers import local2global, plot_3d_interactive, kill_processes
from simulators import Simulator,Gazebo,QGC
from plan import Plan,State
from gcs import GCS

kill_processes()

## Create Plans

In [2]:
gazebo_word_path="simulators/gazebo/worlds/medical_delivery.world" ##
target_box_center=(-85.263400,56.518300, 0.593695, 0)

origin=(0,0,0)
offsets = [ #east, north, up, heading
    target_box_center, 
    (0,-10, 0, 0),
    (1,-15, 0, 0),
    (2,-10, 0, 0)]

n_vehicles=len(offsets)

homes=np.array([offset[:3] for offset in offsets])

altitudes=[5,5,5,7]
plans= [Plan.hover(alt=alt) for alt in altitudes]

## Visualization Parameters

In [3]:
homes=np.array([offset[:3] for offset in offsets])
waypoints = {f'city_homes': {'pos':homes[1:],'color':'blue'},
             f'rural_home': {'pos':homes[[0]],'color':'green'}}
plot_3d_interactive(waypoints,title='Simulation Markers',expand=[0.2,0.2,10],ground=-0.05)

## Choose Simulator

In [4]:
simulator = Simulator(name="NONE",offsets=offsets,plans=plans)

simulator=QGC(offsets=offsets,
              plans=plans,
              origin=(-35.3633245,149.1652241,0,0))

simulator=Gazebo(offsets=offsets,
                 plans=plans,
                 world_path=gazebo_word_path,
                 vehicle_models=n_vehicles*['drone'],
                 markers=waypoints)

Launch Simulator

In [5]:
uavs = simulator.launch()

Vehicle 1 launched 🚀
Vehicle 2 launched 🚀
Vehicle 3 launched 🚀
Vehicle 4 launched 🚀


## Execute Plan

In [ ]:
gcs=GCS(positions=homes.copy())
while gcs.incomplete_missions:
    to_remove = set()
    for id in gcs.incomplete_missions:
        i = id-1
        uav=uavs[i]
        uav.plan.act()
        if uav.plan.state in [State.DONE,State.FAILED]:
            to_remove.add(id)
    gcs.update_missions(to_remove)
    
kill_processes()

In [ ]:
uavs[0].plan

🚀 <Plan 'hover'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  🚀 <Action 'takeoff'>
    🚀 <Step 'takeoff'>
  🕓 <Action 'fly'>